In [1]:
IMAGE_SIZE = 224
NUM_CHANNELS = 3
LMDB_DIR_PATH = "/mnt/data_ssd/lmdb"
MODEL_NAME = "efficientnet_b0"

# Compute Embeddings and Persist them in LMDB

In [2]:
%load_ext google.cloud.bigquery
%load_ext lab_black
%load_ext line_profiler

In [3]:
import lmdb
import pickle
from PIL import Image, ImageFile
import numpy as np
from pympler import asizeof
import torch
from tqdm.notebook import tqdm
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch.nn as nn

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
TRAIN_DATA_FILE = f"{LMDB_DIR_PATH}/seefood_train_data_{MODEL_NAME}"
TEST_DATA_FILE = f"{LMDB_DIR_PATH}/seefood_test_data_{MODEL_NAME}"

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

In [6]:
torch.hub.list("rwightman/gen-efficientnet-pytorch")

Using cache found in /home/mike/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_es',
 'fbnetc_100',
 'mixnet_l',
 'mixnet_m',
 'mixnet_s',
 'mixnet_xl',
 'mnasnet_a1',
 'mnasnet_b1',
 'mobilenetv2_100',
 'mobilenetv2_110d',
 'mobilenetv2_120d',
 'mobilenetv2_140',
 'mobilenetv3_large_100',
 'mobilenetv3_rw',
 'spnasnet_100',
 'tf_efficientnet_b0',
 'tf_efficientnet_b0_ap',
 'tf_efficientnet_b0_ns',
 'tf_efficientnet_b1',
 'tf_efficientnet_b1_ap',
 'tf_efficientnet_b1_ns',
 'tf_efficientnet_b2',
 'tf_efficientnet_b2_ap',
 'tf_efficientnet_b2_ns',
 'tf_efficientnet_b3',
 'tf_efficientnet_b3_ap',
 'tf_efficientnet_b3_ns',
 'tf_efficientnet_b4',
 'tf_efficientnet_b4_ap',
 'tf_efficientnet_b4_ns',
 'tf_efficientnet_b5',
 'tf_efficientnet_b5_ap',
 'tf_efficientnet_b5_ns',
 'tf_efficientnet_b6',
 'tf_efficientnet_b6_ap',
 'tf_efficientnet_b6_ns',
 'tf_efficientnet_b7',
 'tf_efficientnet_b7_ap',
 'tf_efficientnet_b7_ns',
 'tf_efficientnet_b8',
 'tf_efficientnet_b8_ap',
 'tf_effi

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


model = torch.hub.load(
    "rwightman/gen-efficientnet-pytorch", MODEL_NAME, pretrained=True
)
# model = models.mobilenet_v2(pretrained=True)
set_parameter_requires_grad(model, True)

Using cache found in /home/mike/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


## Load Data

In [8]:
%%bigquery df --project zenscr-seefood-dev

SELECT recipe_id, title, image_path, total_calories
FROM `zenscr-seefood-dev.sparkrecipes.base_filtered`
INNER JOIN `zenscr-seefood-dev.sparkrecipes.image_path`
USING (recipe_id)

In [9]:
df

,recipe_id,title,image_path,total_calories
0,70865,Fruit Cocktail Dessert/Salad,../../data/images/70865/000001,30.0
1,70865,Fruit Cocktail Dessert/Salad,../../data/images/70865/000010,30.0
2,70865,Fruit Cocktail Dessert/Salad,../../data/images/70865/000011,30.0
3,70865,Fruit Cocktail Dessert/Salad,../../data/images/70865/000012,30.0
4,70865,Fruit Cocktail Dessert/Salad,../../data/images/70865/000013,30.0
...,...,...,...,...
1311732,418865,Banana Bread Muffins,../../data/images/418865/000005,134.9
1311733,418865,Banana Bread Muffins,../../data/images/418865/000006,134.9
1311734,418865,Banana Bread Muffins,../../data/images/418865/000007,134.9
1311735,418865,Banana Bread Muffins,../../data/images/418865/000008,134.9


In [10]:
df_shuffled = shuffle(df)

In [11]:
df_shuffled

,recipe_id,title,image_path,total_calories
459020,15188,cold crab dip,../../data/images/15188/000007,51.4
875505,361934,Turkey burger cabbage casserole,../../data/images/361934/000009,224.3
932270,70786,Baked Meatloaf and Sides,../../data/images/70786/000005,484.8
31050,22928,Garlic Chicken,../../data/images/22928/000015,311.7
1168791,249005,Cinnamon Vanilla Bran Muffins,../../data/images/249005/000005,115.9
...,...,...,...,...
1231897,367025,cabbage soup,../../data/images/367025/000011,53.5
366248,93809,Berry Stuffed French Toast,../../data/images/93809/000006,347.6
1106199,418506,Multigrain Pancakes,../../data/images/418506/000014,118.3
49026,383436,Easy Italian Chicken Lentil Soup for Two,../../data/images/383436/000003,309.9


## Train-Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    df_shuffled[["image_path"]],
    df_shuffled.total_calories,
    test_size=0.33,
    random_state=42,
)

In [13]:
df_train = X_train.assign(target=y_train).reset_index(drop=True)
df_test = X_test.assign(target=y_test).reset_index(drop=True)

## Write Embeddings to LMDB

In [14]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.model = model
        for p in self.model.parameters():
            p.requires_grad = False

    def forward(self, x):
        x = self.model.features(x)
        x = x.mean([2, 3])
        return x


class Features:
    def __init__(self, image_path, features, target):
        self.shape = features.shape
        self.features = features.numpy().tobytes()
        self.image_path = image_path
        self.target = target.round().item()

    def get_features(self):
        features = np.frombuffer(self.features, dtype=np.float32)
        return torch.from_numpy(features.reshape(self.shape))


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.images = df["image_path"].reset_index(drop=True)
        self.targets = df["target"].reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images.iloc[idx]
        with Image.open(img_path) as f:
            image = f.convert("RGB")
        image = self.transform(image)
        return img_path, image, self.targets.iloc[idx]


transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.CenterCrop(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

dataloader_train = torch.utils.data.DataLoader(
    ImageDataset(df_train, transform), batch_size=64, shuffle=False, num_workers=4
)

dataloader_test = torch.utils.data.DataLoader(
    ImageDataset(df_test, transform), batch_size=64, shuffle=False, num_workers=4
)

In [15]:
feature_extractor = FeatureExtractor(model).to(device)

In [16]:
torch.cuda.empty_cache()

In [17]:
device

device(type='cuda', index=0)

In [18]:
def store_to_lmdb(lmdb_filename, dataloader, image_size=IMAGE_SIZE):
    num_images = len(dataloader.dataset)

    map_size = num_images * (
        300000 + asizeof.asizeof(f"{0:08}") + 2048
    )  # approximate map size

    index = 0
    with lmdb.open(lmdb_filename, map_size=map_size) as env:
        for image_paths, images, targets in tqdm(dataloader):
            with env.begin(write=True) as txn:
                images = images.to(device)
                features = feature_extractor(images).to(cpu)
                for p, f, t in zip(image_paths, features, targets):
                    key = f"{index:08}".encode("ascii")
                    value = Features(p, f, t)
                    txn.put(key, pickle.dumps(value))
                    index += 1

In [19]:
store_to_lmdb(TRAIN_DATA_FILE, dataloader_train)

/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:78

In [20]:
store_to_lmdb(TEST_DATA_FILE, dataloader_test)

/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/mike/.cache/pypoetry/virtualenvs/seefood-nC6BBDGh-py3.8/lib/python3.8/site-packages/PIL/TiffImagePl

KeyboardInterrupt: 